In [1]:
import numpy as np
import math
from numpy import array
import sympy
from sympy import symbols,  cos, sin, pi, simplify, sqrt, atan2, acos, Transpose
from sympy.matrices import Matrix

In [2]:
def get_R0_6(alpha, beta, gamma):
    r11 = cos(alpha) * cos(beta)
    r12 = (cos(alpha)* sin(beta) * sin(gamma)) - (sin(alpha)*cos(gamma))
    r13 = (cos(alpha) * sin(beta) * cos(gamma)) + (sin(alpha)*sin(gamma))
    r21 = sin(alpha) * cos(beta) * 1.0
    r22 = (sin(alpha) * sin(beta) * sin(gamma)) + (cos(alpha) * cos(gamma))
    r23 = (sin(alpha)*sin(beta)*cos(gamma)) - (cos(alpha)*sin(gamma))
    r31 = -sin(beta)
    r32 = cos(beta) * sin(gamma)
    r33 = cos(beta)*cos(gamma)
    return Matrix([[r11, r12, r13],
                   [r21, r22, r23],
                   [r31, r32, r33]])

def find_wrist_center(px, py, pz, roll, pitch, yaw):
    end_effector_length = 0.303
    wx = px - (end_effector_length) * 1.0 * cos(yaw) * cos(pitch)
    wy = py - (end_effector_length) * 1.0 * sin(yaw) * cos(pitch)
    wz = pz - (end_effector_length) * -1.0 * sin(pitch)
    return wx, wy, wz

In [3]:
# Define DH param symbols
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8') # theta_1
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8') # d paramater
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')


# Joint angle symbols
# theta1, theta2, theta3, theta4, theta5, theta6 = 0


# Modified DH params
dh = {alpha0:     0, a0:      0, d1:  0.75,
      alpha1: -pi/2, a1:   0.35, d2:     0, q2: q2 - pi/2,
      alpha2:     0, a2:   1.25, d3:     0,
      alpha3: -pi/2, a3: -0.054, d4:  1.50,
      alpha4:  pi/2, a4:      0, d5:     0,
      alpha5: -pi/2, a5:      0, d6:     0,
      alpha6:     0, a6:      0, d7: 0.303, q7: 0}

# General rotations around axes

R_z = Matrix([[       cos(q1),    -sin(q1),             0, 0],
              [       sin(q1),     cos(q1),             0, 0],
              [             0,           0,             1, 0],
              [             0,           0,             0, 1]])
R_y = Matrix([[       cos(q1),           0,       sin(q1), 0],
              [             0,           1,             0, 0],
              [      -sin(q1),           0,       cos(q1), 0],
              [             0,           0,             0, 1]])
R_x = Matrix([[             1,           0,             0, 0],
              [             0,     cos(q1),      -sin(q1), 0],
              [             0,     sin(q1),       cos(q1), 0],
              [             0,           0,             0, 1]])

# Define Modified DH Transformation matrix
R_corr = simplify(R_z.evalf(subs={q1: pi}) * R_y.evalf(subs={q1: -pi/2}))

In [4]:
T0_1 = Matrix([[            cos(q1),            -sin(q1),            0,              a0],
               [sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
               [sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
               [                  0,                   0,            0,               1]])
T0_1 = T0_1.subs(dh)

T1_2 = Matrix([[            cos(q2),            -sin(q2),            0,              a1],
               [sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
               [sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
               [                  0,                   0,            0,               1]])
T1_2 = T1_2.subs(dh)


T2_3 = Matrix([[            cos(q3),            -sin(q3),            0,              a2],
               [sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
               [sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
               [                  0,                   0,            0,               1]])
T2_3 = T2_3.subs(dh)
T3_4 = Matrix([[            cos(q4),            -sin(q4),            0,              a3],
               [sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
               [sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
               [                  0,                   0,            0,               1]])
T3_4 = T3_4.subs(dh)

T4_5 = Matrix([[            cos(q5),            -sin(q5),            0,              a4],
               [sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
               [sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],
               [                  0,                   0,            0,               1]])
T4_5 = T4_5.subs(dh)

T5_6 = Matrix([[            cos(q6),            -sin(q6),            0,              a5],
               [sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
               [sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],
               [                  0,                   0,            0,               1]])
T5_6 = T5_6.subs(dh)

T6_G = Matrix([[            cos(q7),            -sin(q7),            0,              a6],
               [sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
               [sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
               [                  0,                   0,            0,               1]])
T6_G = T6_G.subs(dh)

In [5]:
T0_2 = simplify(T0_1 * T1_2)
print('Done simplifying T0_2')
T0_3 = simplify(T0_2 * T2_3)
print('Done simplifying T0_3')

Done simplifying T0_2
Done simplifying T0_3


In [6]:
# x,y,z,alpha(yaw),beta(pitch),gamma(roll) = 1.801, 1.005, 1.148,1.557, 0.683, 2.649
# required result : theta1,theta2,theta3,theta4,theta5,theta6 = 0.4,0.2,0.2,1,1,1
x,y,z = 1.801, 1.005, 1.148
yaw, pitch, roll = 1.557, 0.683, 2.649 

# Test
# x, y, z = 2.153, 0.000, 1.947
# roll, pitch, yaw = 0, 0, 0

In [7]:
wx, wy, wz = find_wrist_center(x, y, z, roll, pitch, yaw)

In [8]:
theta1 = atan2(wy, wx)

s1 = sqrt(wx*wx + wy*wy) - dh[a1]
s2 = wz - dh[d1]
s3 = sqrt(s1*s1 + s2*s2)
s4 = sqrt(dh[a3]*dh[a3] + dh[d4]*dh[d4])

wx0, wy0, wz0 = 1.85, 0, 1.947
s1_initial = sqrt(wx0*wx0 + wy0*wy0) - dh[a1]
s2_initial = wz0 - dh[d1]
s3_initial = sqrt(s1_initial*s1_initial + s2_initial*s2_initial)
s4_initial = s4

beta1_initial = atan2(s2_initial, s1_initial)
beta2_d_initial = (dh[a2]*dh[a2] + s3_initial*s3_initial - s4_initial*s4_initial) / (2.0*dh[a2]*s3_initial)
beta2_initial = atan2(sqrt(1 - beta2_d_initial*beta2_d_initial), beta2_d_initial)

beta1 = atan2(s2, s1)
beta2_d = (dh[a2]*dh[a2] + s3*s3 - s4*s4) / (2*dh[a2]*s3)
beta2 = atan2(sqrt(1 - beta2_d*beta2_d), beta2_d)

print s1, s2, s3, s4
print(beta1)
print(beta2)

# theta2
print(beta1_initial, beta2_initial)
print(beta1, beta2)
theta2 = (beta1_initial + beta2_initial) - (beta1 + beta2)

# Finding theta3
# beta3_intiial
beta4 = pi/2 - atan2(dh[a3], dh[d4])
beta3_initial_d = (dh[a2]*dh[a2] + s4_initial*s4_initial - s3_initial*s3_initial) / (2.0*dh[a2]*s4_initial) 
beta3_initial = atan2(sqrt(1 - beta3_initial_d*beta3_initial_d), beta3_initial_d)
# beta3_initial = 1.5348118667

# beta3
beta3_d = (dh[a2]*dh[a2] + s4*s4 - s3*s3) / (2.0*dh[a2]*s4)
beta3 = atan2(sqrt(1 - beta3_d*beta3_d), beta3_d)

# theta3
theta3 = beta3_initial - beta3

print("thetas")
print theta1, theta2, theta3

1.60571404854478 0.589230241460075 1.71041219685349 1.50097168527591
0.351701920260096
1.02077785025322
(0.673520239704882, 0.897304620098513)
(0.351701920260096, 1.02077785025322)
thetas
0.404667584932112 0.198345089290080 0.204089580864693


In [9]:
R0_6 = get_R0_6(yaw, pitch, roll)
print(R0_6)
R0_6 = R0_6.row_join(Matrix([[x],
                             [y],
                             [z]]))
R0_6 = R0_6.col_join(Matrix([[0, 0, 0, 1]]))

# Get R0_3
R0_3 = (T0_1 * T1_2 * T2_3).evalf(subs={q1: theta1, q2: theta2, q3: theta3})

# Get R3_6
var_R3_6 = T3_4 * T4_5 * T5_6
R3_6 = Transpose(R0_3) * R0_6
print(var_R3_6)
print(R3_6)

r23 = R3_6[1, 2]
r33 = R3_6[2, 2]
theta4 = -atan2(r33, r23)

theta5 = acos(r23)

r21 = R3_6[1, 0]
r22 = R3_6[1, 1]
theta6 = atan2(r22, r21)
if math.isnan(theta4):
    theta4 = 0
else:
    theta4 = theta4.evalf()
if math.isnan(theta5):
    theta5 = 0
else:
    theta5 = theta5.evalf()
if math.isnan(theta6):
    theta6 = 0
else:
    theta6 = theta6.evalf()
print theta4, theta5, theta6


Matrix([[0.0107012345132777, 0.885143468653330, 0.465195145589723], [0.775609023398699, 0.286281429692164, -0.562559850892265], [-0.631122909109160, 0.366829637452813, -0.683461696574014]])
Matrix([[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4), -0.0540000000000000], [sin(q5)*cos(q6), -sin(q5)*sin(q6), cos(q5), 1.50000000000000], [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4), sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6), sin(q4)*sin(q5), 0], [0, 0, 0, 1]])
Matrix([
[-0.457249729874093,   0.700343934958169, -0.548125038013569,  1.85966699377873],
[ 0.537208207807252,   0.708686537973418,  0.457351869308273,    1.437723984252],
[ 0.708752443378214, -0.0853332506269317, -0.700277238201309, 0.214751990343599],
[ -1.05881935819495,     1.2770696071745,  -1.22725295238973,  4.49102679249649]])
0.992244124511953 1.09578123369778 0.922173508785066
